In [ ]:
!alias python3==python3.6.9

In [1]:
from calendar import monthrange
from datetime import datetime,timedelta
#
#import pandas as pd
import numpy as np

import os
import time
import pickle
from concurrent.futures import ThreadPoolExecutor

vladivostok=(43.1198,131.8869)
chennai=(13.0827, 80.2707)
#print(datetime.now())
latsrange=(-15,45)
lonsrange=(75,135)
from joblib import Parallel, delayed
import sklearn
import pickle
from math import sin,cos,pi,floor,ceil
def lat_length(lat_deg):
    lat=lat_deg*pi/180
    return 111132.92-559.82*cos(2*lat)+1.175*cos(4*lat)-0.0023*cos(6*lat)
def lon_length(lat_deg):
    lat=lat_deg*pi/180
    return 111412.84*cos(lat)-93.5*cos(3*lat)+0.118*cos(5*lat)

def Grid_generation(res=0.08):
    Grid=[]
    
    n_steps=int((lonsrange[1]-lonsrange[0])/res)
    ref_lat=max(0,min(latsrange[0],latsrange[1]))

    lon_stepsize=res*lon_length(ref_lat)
    lat_stepsize=res*lat_length(ref_lat)
    next_lat=latsrange[0]
    while(next_lat<=latsrange[1]): 
        diff=(lon_length(ref_lat)-lon_length(next_lat))*(lonsrange[1]-lonsrange[0])
        start=lonsrange[0]-((.5*diff)/lon_length(next_lat))
        lonstep_lat=lon_stepsize/lon_length(next_lat)
        cur_lon=start

        lonlist=[(next_lat,cur_lon)]
        for i in range(n_steps):
            cur_lon+=lonstep_lat
            lonlist.append((next_lat,cur_lon))
        Grid.append(lonlist)
        next_lat=round(next_lat+res,2)
    return np.asarray(Grid)

res=0.04
Grid=Grid_generation(res)

land=np.load("land_0.04.npy")
land2=np.empty(land.shape)
for i in range(land.shape[0]):
    for j in range(land.shape[1]):
        if(i> 0 and j>0 and i<land.shape[0]-1 and j<land.shape[1]-1):
            land2[i][j]=(((land[i-1][j]+land[i+1][j]+land[i][j-1]+land[i][j+1]+land[i-1][j+1]+land[i-1][j-1]+land[i+1][j-1]+land[i+1][j+1])/8)+land[i][j])/2
        else:
            land2[i][j]=land[i][j]
land_ind=np.asarray(np.where(land2>0))


grid_res=(len(Grid),len(Grid[0]))

minlon,maxlon=min(Grid[-1][0][1],Grid[0][0][1]),max(Grid[-1][-1][1],Grid[0][-1][1])
lons=(minlon,maxlon)
lats=(latsrange[0],latsrange[1])
lons,lats

heightall=np.load("height.npy").tolist()
winds_u=np.load("winds_u.npy").tolist()
winds_v=np.load("winds_v.npy").tolist()
windlats=np.load("windlats.npy")
windlons=np.load("windlons.npy")
windlatsmap={item:i for i,item in enumerate(windlats)}
windlonsmap={item:i for i,item in enumerate(windlons)}



folder="0/water/"



waterlats=np.load(folder+"lats.npy")
waterlats=[round(i,2) for i in waterlats]
waterlatsmap={item:i for i,item in enumerate(waterlats)}



waterlons=np.load(folder+"lons_0.npy")
waterlons=[round(i,2) for i in waterlons]
waterlonsmap={item:i for i,item in enumerate(waterlons)}
turl="https://tds.hycom.org/thredds/dodsC/GLBv0.08/expt_93.0/uv3z.dods?time"
time_list=list(np.load("time_list.npy"))
timesmap={t:i for i,t in enumerate(time_list)}


from scipy.interpolate import LinearNDInterpolator
folder="0/water/"
time_res_mins=4
wind_u=[]
wind_v=[]
water_u=[]
water_v=[]
waveheight=[]
windall_vars=[]
waterall_vars=[]

# with open("model.m","rb") as file:
#     model=pickle.load(file)

timesrange=["2018-01-02 00:00:00","2018-01-16 00:00:00"]
ref_time="2000-01-01 00:00:00"
total_iters=(datetime.strptime(timesrange[1],"%Y-%m-%d %H:%M:%S")-datetime.strptime(timesrange[0],"%Y-%m-%d %H:%M:%S")).total_seconds()/(60*time_res_mins)
initial_offset=(datetime.strptime(timesrange[0],"%Y-%m-%d %H:%M:%S")-datetime.strptime(ref_time,"%Y-%m-%d %H:%M:%S")).total_seconds()/3600
dwnld_offset=initial_offset-24
#print(total_iters,initial_offset)

#@numba.jit
def prev_nxt(x,resolution):
    if(x>0):
        return (round(int(x/resolution)*resolution,2),round((int(x/resolution)+1)*resolution,2))
    else:
        return (round((int(x/resolution)-1)*resolution,2),round((int(x/resolution))*resolution,2))


In [2]:
narrowband=np.zeros((Grid.shape[0],Grid.shape[1]))

for i in range(30,100):
    for j in range(400,1000):
        narrowband[i][j]=1
a=np.where(narrowband>0)
nb_pts=np.asarray([(a[0][i],a[1][i]) for i in range(len(a[0]))])
nb_pts=nb_pts.astype(np.int32)

print(datetime.now())

2020-04-28 10:35:08.204314


In [ ]:
def create_gridfile(t,nb):
    #nb=np.asarray(nb)
    s=time.time()
    
    
    Wa_u=[[0]*Grid.shape[1] for _ in range(Grid.shape[0])] 
    Wa_v=[[0]*Grid.shape[1] for _ in range(Grid.shape[0])] 
    
    a=np.where(nb>0)
    nb_pts=[(a[0][i],a[1][i]) for i in range(len(a[0]))]


    idate=datetime.strptime(timesrange[0],"%Y-%m-%d %H:%M:%S")+timedelta(minutes=time_res_mins*t)
    imonth=idate.month
    iyear=idate.year
    ihrs_from_month_beginning=(idate-datetime.strptime("{}-{}-01 00:00:00".format(iyear,imonth),"%Y-%m-%d %H:%M:%S")).total_seconds()/3600
    #print(imonth,idate,ihrs_from_month_beginning)
    ihrs=initial_offset+(t*time_res_mins/60)
    prev_t,next_t=prev_nxt(ihrs,3)
    try:
        prev_ind=timesmap[prev_t]
        next_ind=timesmap[next_t]
    except:
        close_val=min(timesmap.keys(), key=lambda x:abs(x-ihrs))
        if(close_val<ihrs):
            prev_ind=timesmap[close_val]
            next_ind=prev_ind+1
            prev_t=close_val
            next_t=time_list[next_ind]
        else:
            next_ind=timesmap[close_val]
            prev_ind=next_ind-1
            prev_t=time_list[prev_ind]
            next_t=close_val
    prev_wateruv_file=np.load(folder+"{}_0.npy".format(prev_ind)).tolist()
    next_wateruv_file=np.load(folder+"{}_0.npy".format(next_ind)).tolist()
    def each_interp(pt):
        #print(i,j)
        i,j=pt 
        # if nb[i][j]==1:
        #     print(i,j)
        ilat,ilon=Grid[i][j]

        try:
            variables=[]
            fdata=[]
            for hr in [prev_t,next_t]:
                for la in prev_nxt(ilat,0.5):
                    for lo in prev_nxt(ilon,0.5):
                        hr_ind=int((hr-dwnld_offset)/3)
                        la_ind=windlatsmap[la]
                        lo_ind=windlonsmap[lo]

                        if(hr==prev_t):
                            uv_file=prev_wateruv_file
                        else:
                            uv_file=next_wateruv_file

                        variables.append((hr,la,lo))
                        fdata.append((heightall[hr_ind][la_ind][lo_ind],winds_u[hr_ind][la_ind][lo_ind],winds_v[hr_ind][la_ind][lo_ind],0.001*uv_file[0][0][la_ind][lo_ind],0.001*uv_file[1][0][la_ind][lo_ind]))
            #variables=[[prev_t,next_t],prev_nxt(ilat,0.5),prev_nxt(ilon,0.5)]
            interpolator=LinearNDInterpolator(variables,fdata,rescale=True)
            h,wi_u,wi_v,wa_u,wa_v=interpolator(ihrs,ilat,ilon)
#             wa_u=0.001*wa_u
#             wa_v=0.001*wa_v

        except Exception as e:
            print(e)
            h,wi_u,wi_v,wa_u,wa_v=[0,0,0,0,0]
        return h,wi_u,wi_v,wa_u,wa_v
    model_input=Parallel(n_jobs=1)(delayed(each_interp)(pt) for pt in nb_pts)
    
    output=model.predict(model_input)
    F=np.zeros((Grid.shape[0],Grid.shape[1]))
    
    for k,(i,j) in enumerate(nb_pts):
        F[i][j]=output[k]
        Wa_u[i][j]=model_input[k][3]
        Wa_v[i][j]=model_input[k][4]
    

    F[land_ind[0],land_ind[1]]=0
    Wa_u[land_ind[0],land_ind[1]]=0
    Wa_v[land_ind[0],land_ind[1]]=0
    
    #Wa_u,Wa_v=out[:,3],out[:,4]
#     for k,pt in enumerate(nb_pts):
#         Wa_u[i][j]=model_input[k][3]
#         Wa_v[i][j]=model_input[k][4]
    return F,Wa_u,Wa_v


In [ ]:
m=time.time()
s=create_gridfile(0,nb_pts)
time.time()-m

In [ ]:
split_nb=[nb_pts[i:i+int(ceil(len(nb_pts)/4))] for i in range(0,len(nb_pts),int(ceil(len(nb_pts)/4)))]

In [ ]:
from scipy.interpolate import LinearNDInterpolator


In [ ]:
def trilinear_interpolation(points,values,outpoint):
    x,y,z=outpoint
    x0,x1=points[0]
    y0,y1=points[1]
    z0,z1=points[2]
    c000,c001,c010,c011,c100,c101,c110,c111=values
    denom=(x1-x0)*(y1-y0)*(z1-z0)
    a0=(c000*x1*y1*z1-c001*x1*y1*z0-c010*x1*y0*z1+c011*x1*y0*z0-c100*x0*y1*z1+c101*x0*y1*z0+c110*x0*y0*z1-c111*x0*y0*z0)/denom
    a1=(-c000*y1*z1+c001*y1*z0+c010*y0*z1-c011*y0*z0+c100*y1*z1-c101*y1*z0-c110*y0*z1+c111*y0*z0)/denom
    a2=(-c000*x1*z1+c001*x1*z0+c010*x1*z1-c011*x1*z0+c100*x0*z1-c101*x0*z0-c110*x0*z1+c111*x0*z0)/denom
    a3=(-c000*x1*y1+c001*x1*y1+c010*x1*y0-c011*x1*y0+c100*x0*y1-c101*x0*y1-c110*x0*y0+c111*x0*y0)/denom
    a4=(c000*z1-c001*z0-c010*z1+c011*z0-c100*z1+c101*z0+c110*z1-c111*z0)/denom
    a5=(c000*y1-c001*y1-c010*y0+c011*y1-c100*y1+c101*y1+c110*y0-c111*y0)/denom
    a6=(c000*x1-c001*x1-c010*x1+c011*x0-c100*x0+c101*x0+c110*x0-c111*x0)/denom
    a7=(-c000+c001+c010-c011+c100-c101-c110+c111)/denom
    return a0+a1*x+a2*y+a3*z+a4*x*y+a5*x*z+a6*y*z+a7*x*y*z

In [ ]:
def interp_3d(points,values_list,outpoint):
    output=[]
    for i in range(len(values_list[0])):
        output.append(trilinear_interpolation(points,[val[i] for val in values_list],outpoint))
    return output

In [ ]:
my_interpolating_function = RegularGridInterpolator((x, y, z), data)


In [ ]:
points=[(0,1),(0,1),(0,1)]
points2=[(0,0,0),(0,0,1),(0,1,0),(0,1,1),(1,0,0),(1,0,1),(1,1,0),(1,1,1)]
values=[0,1,1,4,1,4,4,9]
outpt=[0.5,0.5,0.5]
print(trilinear_interpolation(points,values,outpt))
LinearNDInterpolator(points2,values)(0.5,0.5,0.5)

In [ ]:
s=time.time()
m=create_gridfile(0,nb_pts)
print(time.time()-s)

In [3]:
from ctypes import *

In [4]:
lib=CDLL("/home/revanth/Mtech/bp/cpp2/openmp4/interpolation/libinter.so.1")

In [ ]:
a=[1,2,3,4,5]
b=[a]*3
arr = (c_double * len(a))(*a)


In [5]:
def double_array(M,N,types=c_double):
    double_P = POINTER(types)
    inner_array = (types * N)
    inner_array_P = POINTER(inner_array)
    dest = (double_P * M) ()
    for i in range(M):
        dest[i] = inner_array()
    return dest
def triple_array(P,M,N):
    double_P_P=POINTER(POINTER(c_double))
    dest=(double_P_P*P)()
    for i in range(P):
        dest[i]=double_array(M,N)
    return dest
def copy_dlist(li,types=c_double):
    M,N=len(li),len(li[0])
    dest=double_array(M,N,types)
    for i in range(M):
        for j in range(N):
            if(types==c_int):
                dest[i][j]=int(li[i][j])
            else:
                dest[i][j]=li[i][j]
    return dest
def copy_tlist(li):
    P,M,N=len(li),len(li[0]),len(li[0][0])
    dest=triple_array(P,M,N)
    for i in range(P):
        for j in range(M):
            for k in range(N):
                dest[i][j][k]=li[i][j][k]
    return dest

In [6]:
c_heightall=copy_tlist(heightall)
c_winds_u=copy_tlist(winds_u)
c_winds_v=copy_tlist(winds_v)
c_Grid=copy_tlist(Grid)
c_wilatsmap=copy_dlist([[item,i] for i,item in enumerate(windlats)])
c_wilonsmap=copy_dlist([[item,i] for i,item in enumerate(windlons)])

In [7]:
t=0
idate=datetime.strptime(timesrange[0],"%Y-%m-%d %H:%M:%S")+timedelta(minutes=time_res_mins*t)
imonth=idate.month
iyear=idate.year
ihrs_from_month_beginning=(idate-datetime.strptime("{}-{}-01 00:00:00".format(iyear,imonth),"%Y-%m-%d %H:%M:%S")).total_seconds()/3600
#print(imonth,idate,ihrs_from_month_beginning)
ihrs=initial_offset+(t*time_res_mins/60)
prev_t,next_t=prev_nxt(ihrs,3)
try:
    prev_ind=timesmap[prev_t]
    next_ind=timesmap[next_t]
except:
    close_val=min(timesmap.keys(), key=lambda x:abs(x-ihrs))
    if(close_val<ihrs):
        prev_ind=timesmap[close_val]
        next_ind=prev_ind+1
        prev_t=close_val
        next_t=time_list[next_ind]
    else:
        next_ind=timesmap[close_val]
        prev_ind=next_ind-1
        prev_t=time_list[prev_ind]
        next_t=close_val
prev_wateruv_file=np.load(folder+"{}_0.npy".format(prev_ind))
next_wateruv_file=np.load(folder+"{}_0.npy".format(next_ind))


In [9]:
t=time.time()
prev_wateruv=np.asarray([prev_wateruv_file[0][0],prev_wateruv_file[1][0]])
prev_wateruv=prev_wateruv.astype(np.float64)
# c_prevuv2 = (c_double*prev_wateruv.size)(*prev_wateruv.flatten().tolist())
c_prevuv = prev_wateruv.ctypes.data_as(c_double_p)

next_wateruv=np.asarray([next_wateruv_file[0][0],next_wateruv_file[1][0]])
next_wateruv=next_wateruv.astype(np.float64)
# c_nextuv2=(c_double*next_wateruv.size)(*next_wateruv.flatten().tolist())
c_nextuv=next_wateruv.ctypes.data_as(c_double_p)

time.time()-t

0.009538888931274414

In [8]:
c_double_p =POINTER(c_double)
c_int_p=POINTER(c_int)

In [ ]:
s=time.time()
c_narrowband2=(c_int*nb_pts.size)(*nb_pts.flatten().tolist())
print(time.time()-s)
# c_prevuv=copy_tlist(prev_wateruv)
# c_nextuv=copy_tlist(next_wateruv)
c_prevuv2 = (c_double*prev_wateruv.size)(*prev_wateruv.flatten().tolist())
c_nextuv2=(c_double*next_wateruv.size)(*next_wateruv.flatten().tolist())
output=double_array(len(nb_pts),5)
time.time()-s

In [53]:
s=time.time()
c_narrowband=nb_pts.ctypes.data_as(c_int_p)
print(time.time()-s)
# c_prevuv=copy_tlist(prev_wateruv)
# c_nextuv=copy_tlist(next_wateruv)
# c_prevuv = prev_wateruv.ctypes.data_as(c_double_p)
# c_nextuv=next_wateruv.ctypes.data_as(c_double_p)
output=(c_double*5*len(nb_pts))()
time.time()-s

0.000127077102661


0.0071849822998046875

In [54]:
t=time.time()
lib.interpolation(c_narrowband,c_int(len(nb_pts)),c_wilatsmap,
            c_int(len(windlats)),c_wilonsmap,c_int(len(windlons)),
                 c_int(int(dwnld_offset)),c_double(ihrs),c_double(prev_t),c_double(next_t),
                 c_Grid,c_heightall,c_winds_u,c_winds_v,c_prevuv,c_nextuv,
                  c_int(next_wateruv.shape[0]),c_int(next_wateruv.shape[1]),c_int(next_wateruv.shape[2]),output)
time.time()-t

0.4169461727142334

In [55]:
t=time.time()
#b=np.empty((len(nb_pts),5))
b=np.copy(output,"C")
time.time()-t

0.010040044784545898

In [ ]:
outputpt=(c_double*3)(0.5,0.5,0.5)
points=double_array(3,2)
points[0][0],points[0][1]=0,1
points[1][0],points[1][1]=0,1
points[2][0],points[2][1]=0,1
size=5
values=double_array(8,5)
for i in range(8):
    values[i][0],values[i][1],values[i][2],values[i][3],values[i][4]=0,1,2,3,4
    #
out=(c_double*size)(0,0,0,0,0)

In [56]:
output[10][3],b[10][3]

(-0.022048264258668238, -0.022048264258668238)

In [ ]:
output[0][3]

In [ ]:
heightall[0][0][6]

In [ ]:
heightall=np.load("height.npy")

In [ ]:
heightall=np.asarray(heightall)

In [ ]:
heightall[5][8][159],heightall.shape

In [ ]:
c_heightall[5][8][159]

In [ ]:
c_float_p =POINTER(c_float)
data = np.array([[0.1, 0.1], [0.2, 0.2], [0.3, 0.3]])
data = data.astype(np.float32)
data_p = data.ctypes.data_as(c_float_p)

In [ ]:
data_p[3]

In [ ]:
c_double_p =POINTER(c_double)
prev_wateruv_file=prev_wateruv_file.astype(np.float64)
data_p = prev_wateruv_file.ctypes.data_as(c_double_p)

In [ ]:
data_p[1725680+100]

In [ ]:
prev_wateruv_file.shape[0]*prev_wateruv_file.shape[1]*prev_wateruv_file.shape[2]*prev_wateruv_file.shape[3]

In [ ]:
diff=uvv-prev_wateruv_file

In [ ]:
diff[diff!=0]